In [ ]:
!pip install wordninja
!pip install scattertext && python -m spacy.en.download
!pip install sklearn

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_columns', 100)
sns.set_style("darkgrid")

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

import wordninja

%matplotlib inline
import scattertext as st
import re, io
from pprint import pprint
from scipy.stats import rankdata, hmean, norm
import spacy
import os, pkgutil, json, urllib
from urllib.request import urlopen
from IPython.display import IFrame
from IPython.core.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer
display(HTML("<style>.container { width:98% !important; }</style>"))

/usr/bin/python3: Error while finding module specification for 'spacy.en.download' (ModuleNotFoundError: No module named 'spacy.en')


In [ ]:
model_data = pd.read_csv('../data/fin_combined_data.csv', keep_default_na=False)

In [ ]:
model_data.head(3)

,title,selftext,is_suicide,url
0,Our most-broken and least-understood rules is ...,We understand that most people who reply immed...,1,https://www.reddit.com/r/depression/comments/d...
1,Regular Check-In Post,Welcome to /r/depression's check-in post - a p...,1,https://www.reddit.com/r/depression/comments/e...
2,I hate it so much when you try and express you...,I've been feeling really depressed and lonely ...,1,https://www.reddit.com/r/depression/comments/f...


In [ ]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2516 entries, 0 to 2515
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       2516 non-null   object
 1   selftext    2516 non-null   object
 2   is_suicide  2516 non-null   int64 
 3   url         2516 non-null   object
dtypes: int64(1), object(3)
memory usage: 78.8+ KB


In [ ]:
model_data['is_suicide'].mean()
#baseline accuracy of 60.25%

0.6025437201907791

In [ ]:
def multi_modelling(columns_list, model):
    for i in columns_list:
        X = model_data[i]
        y = model_data['is_suicide']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
        
        cvec = CountVectorizer()
        cvec.fit(X_train)
        
        X_train = pd.DataFrame(cvec.transform(X_train).todense(),
                               columns=cvec.get_feature_names())
        X_test = pd.DataFrame(cvec.transform(X_test).todense(),
                               columns=cvec.get_feature_names())
        
        nb = MultinomialNB()
        nb.fit(X_train,y_train)
        
        pred = nb.predict(X_test)
        
        tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
        
        
        nb.predict_proba(X_test)
        pred_proba = [i[1] for i in nb.predict_proba(X_test)] 
        auc = roc_auc_score(y_test, pred_proba)

        classi_dict = (classification_report(y_test,pred, output_dict=True))

        model_results = {}
        model_results['series used (X)'] = i
        model_results['model'] = model
        model_results['AUC Score'] = auc
        model_results['precision']= classi_dict['weighted avg']['precision']
        model_results['recall (sensitivity)']= classi_dict['weighted avg']['recall']
        model_results['confusion matrix']={"TP": tp,"FP":fp, "TN": tn, "FN": fn}
        model_results['train accuracy'] = nb.score(X_train, y_train)
        model_results['test accuracy'] = nb.score(X_test, y_test)
        model_results['baseline accuracy']=0.5166
        model_results['specificity']= tn/(tn+fp)  
        model_results['f1-score']= classi_dict['weighted avg']['f1-score']

        model_results
        df_list.append(model_results) 

    pd.set_option("display.max_colwidth", 50)
    return (pd.DataFrame(df_list)).round(2)

In [ ]:
columns_list = ['title', 'selftext']
model = "CountVec + MultinomialNB"
df_list=[]
multi_modelling(columns_list, model)

,series used (X),model,AUC Score,precision,recall (sensitivity),confusion matrix,train accuracy,test accuracy,baseline accuracy,specificity,f1-score
0,title,CountVec + MultinomialNB,0.96,0.89,0.89,"{'TP': 363, 'FP': 54, 'TN': 196, 'FN': 16}",0.96,0.89,0.52,0.78,0.89
1,selftext,CountVec + MultinomialNB,0.99,0.94,0.94,"{'TP': 377, 'FP': 38, 'TN': 212, 'FN': 2}",0.96,0.94,0.52,0.85,0.94


In [ ]:
X = model_data["title"]
y = model_data['is_suicide']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

tvec_optimised = TfidfVectorizer(max_df= 0.5, max_features=70, min_df=2, ngram_range=(1, 3),stop_words = 'english')
X_train_tvec = tvec_optimised.fit_transform(X_train).todense()
X_test_tvec = tvec_optimised.transform(X_test).todense()

nb = MultinomialNB()
nb.fit(X_train_tvec, y_train)
accuracy = nb.score(X_test_tvec, y_test)

pred_proba = [i[1] for i in nb.predict_proba(X_test_tvec)] 
auc = roc_auc_score(y_test, pred_proba)

print("ACCURACY: {}\nAUC SCORE: {}".format(accuracy, auc))


ACCURACY: 0.7376788553259142
AUC SCORE: 0.8325751978891821


In [ ]:
X = model_data["selftext"]
y = model_data['is_suicide']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

tvec_optimised = TfidfVectorizer(max_df= 0.5, max_features=70, min_df=2, ngram_range=(1, 3),stop_words = 'english')
X_train_tvec = tvec_optimised.fit_transform(X_train).todense()
X_test_tvec = tvec_optimised.transform(X_test).todense()

nb = MultinomialNB()
nb.fit(X_train_tvec, y_train)
accuracy = nb.score(X_test_tvec, y_test)

pred_proba = [i[1] for i in nb.predict_proba(X_test_tvec)] 
auc = roc_auc_score(y_test, pred_proba)

print("ACCURACY: {}\nAUC SCORE: {}".format(accuracy, auc) )

ACCURACY: 0.821939586645469
AUC SCORE: 0.9560105540897098


In [ ]:
model_data2 = pd.read_csv('../data/testdata.csv', keep_default_na=False)

In [ ]:
model_data2['is_suicide'].mean()
#baseline accuracy of 65.18%

0.6518010291595198

In [ ]:
X = model_data2["title"]
y = model_data2['is_suicide']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

tvec_optimised = TfidfVectorizer(max_df= 0.5, max_features=70, min_df=2, ngram_range=(1, 3),stop_words = 'english')
X_train_tvec = tvec_optimised.fit_transform(X_train).todense()
X_test_tvec = tvec_optimised.transform(X_test).todense()

nb = MultinomialNB()
nb.fit(X_train_tvec, y_train)
accuracy = nb.score(X_test_tvec, y_test)

pred_proba = [i[1] for i in nb.predict_proba(X_test_tvec)] 
auc = roc_auc_score(y_test, pred_proba)

print("ACCURACY: {}\nAUC SCORE: {}".format(accuracy, auc))

ACCURACY: 0.8082191780821918
AUC SCORE: 0.8661506707946337


In [ ]:
X = model_data2["selftext"]
y = model_data2['is_suicide']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

tvec_optimised = TfidfVectorizer(max_df= 0.5, max_features=70, min_df=2, ngram_range=(1, 3),stop_words = 'english')
X_train_tvec = tvec_optimised.fit_transform(X_train).todense()
X_test_tvec = tvec_optimised.transform(X_test).todense()

nb = MultinomialNB()
nb.fit(X_train_tvec, y_train)
accuracy = nb.score(X_test_tvec, y_test)

pred_proba = [i[1] for i in nb.predict_proba(X_test_tvec)] 
auc = roc_auc_score(y_test, pred_proba)

print("ACCURACY: {}\nAUC SCORE: {}".format(accuracy, auc) )

ACCURACY: 0.8356164383561644
AUC SCORE: 0.9330237358101136


In [ ]:
pip install praw

     |████████████████████████████████| 153kB 2.8MB/s 
     |████████████████████████████████| 204kB 8.7MB/s 


In [ ]:
import praw
import os
import sys
from praw.exceptions import APIException
import random as r
import time as t
import csv

In [ ]:
userAgent = 'Ada'
cID = 'I2YE0Oa0aIV57Q'
cSC= '7lyGmWG8NL7w2DSMCdx8tqkHiOE'
userN = 'AdaTheChatbot'
userP ='thestillsmoking$'
reddit = praw.Reddit(user_agent=userAgent, client_id=cID, client_secret=cSC, username=userN, password=userP)


In [ ]:
subreddit = reddit.subreddit('testingground4bots+SuicideWatch+depression+rant')
keywords = ['pineapples']
bot_phrase = ['ily', 'you are loved', 'your life matters', 'it will be better soon', 'you deserve the world', 'you deserve to be happy', 'you are such a huge inspiration to many of us', 'you are beautiful inside and out', 'never lose hope', 'somewhere out there, someone cares about you', 'you are worthy', "i may not understand what you're going through, but i believe in you", 'the sun will still rise']
duplicateComments = []
duplicateSubmissions = []

def output():
  item_url = 'https://www.reddit.com' + submission.permalink
  print("URL: ", item_url)
  print("BODY: ", submission.selftext)
  if type_to == 'title':
    print("TITLE: ", submission.title)
  elif type_to == 'body':
    print("BODY2: ", submission.selftext)
  elif type_to == 'comment':
    print("COMMENT: ", comment.body)


In [ ]:
type_to = 'body'
b_confirm = False

type_to = 'title'
t_confirm = False

for submission in subreddit.new(limit=10):
  n_body = submission.selftext.lower()
  if n_body == "":
    n_body = " "
  n_title = submission.title.lower()
  author = str(submission.author)
  s_id = submission.id
  s_url = 'https://www.reddit.com' + submission.permalink

  with open('reddit.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["title", "selftext", "url"])
    writer.writerow([n_title, n_body, s_url])
    file.close()
    redditdata = pd.read_csv('../content/reddit.csv', keep_default_na=False)

  if s_id in duplicateSubmissions:
    print("duplicate")
  else:
    duplicateSubmissions.append(s_id)
    print('Current post: https://www.reddit.com' + submission.permalink)
    X_train = model_data["title"]
    y_train = model_data['is_suicide']
    X_test = redditdata["title"]
    
    tvec_optimised = TfidfVectorizer(max_df= 0.5, max_features=70, min_df=2, ngram_range=(1, 3),stop_words = 'english')
    X_train_tvec = tvec_optimised.fit_transform(X_train).todense()
    X_test_tvec = tvec_optimised.transform(X_test).todense()
    nb = MultinomialNB()
    nb.fit(X_train_tvec, y_train)
    predictions_t = nb.predict(X_test_tvec)

    X_train = model_data["selftext"]
    y_train = model_data['is_suicide']
    X_test = redditdata["selftext"]

    tvec_optimised = TfidfVectorizer(max_df= 0.5, max_features=70, min_df=2, ngram_range=(1, 3),stop_words = 'english')
    X_train_tvec = tvec_optimised.fit_transform(X_train).todense()
    X_test_tvec = tvec_optimised.transform(X_test).todense()
    nb = MultinomialNB()
    nb.fit(X_train_tvec, y_train)
    predictions_b = nb.predict(X_test_tvec)

      # RELEVANT CODE STARTS HERE - if either Body or Title returned 1 from the model, reply with a random phrase
    if predictions_t == 1 or predictions_b == 1:
      try:
        message = r.choice(bot_phrase)
        submission.reply(message)
        output()

        #print("TARG: ",type(str(author)))
        #reddit.redditor(author).message("TEST", "Testing.")
        t.sleep(5)
      except APIException as e:
        print("\nComment not sent.\n----------------------\n")
    else:
      print("Not Depressed.", duplicateSubmissions)

Current post: https://www.reddit.com/r/testingground4bots/comments/jhsvq6/maybe_i_should_end_it_all/
URL:  https://www.reddit.com/r/testingground4bots/comments/jhsvq6/maybe_i_should_end_it_all/
BODY:  hahaha... jokes
TITLE:  maybe i should end it all


comments?